# 第四讲 功能扩展

## 一、 回顾

### 1、tf.keras 搭建神经网络八股——六步法

### 2、代码 mnist_train_baseline.py

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test), validation_freq=1)
model.summary()

## 二、本讲用 tf.keras 完善功能模块

### 1、自制数据集，应对特定应用

#### 1.1、观察数据集数据结构，配成特征标签对

In [5]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os

train_path = "./MNIST_FC/mnist_image_label/mnist_train_jpg_60000/"
train_txt = "./MNIST_FC/mnist_image_label/mnist_train_jpg_60000.txt"
x_train_save_path = "./MNIST_FC/mnist_image_label/mnist_x_train.npy"
y_train_save_path = "./MNIST_FC/mnist_image_label/mnist_y_train.npy"


test_path = "./MNIST_FC/mnist_image_label/mnist_test_jpg_10000/"
test_txt = "./MNIST_FC/mnist_image_label/mnist_test_jpg_10000.txt"
x_test_save_path = "./MNIST_FC/mnist_image_label/mnist_x_test.npy"
y_test_save_path = "./MNIST_FC/mnist_image_label/mnist_y_test.npy"

def generateds(path, txt):
    f = open(txt, 'r') #以只读形式打开txt文件
    contents = f.readlines() # 读取文件中所有行
    f.close() # 关闭文件
    x, y_ = [], []
    for content in contents: #逐行取出
        value = content.split() # 以空格分，开图片路径为value[0] , 标签为value[1] , 存入列表
        img_path = path + value[0] #拼出图片路径和文件名
        img = Image.open(img_path) # 读入图片
        img = np.array(img.convert('L'))  # 图片变为8位宽灰度值的np.array格式
        img = img/ 255.  # 数据归一化 （实现预处理）
        x.append(img)  # 归一化后的数据，贴到列表x
        y_.append(value[1]) # 标签贴到列表y_
        # print('loading :' + content)  # 打印状态提示
    x = np.array(x) # 变为np.array格式
    y_ = np.array(y_) # 变为np.array格式
    y_ = y_.astype(np.int64) # 变为64位整型
    return x, y_


if (os.path.exists(x_train_save_path)
    and os.path.exists(y_train_save_path) 
    and os.path.exists(x_test_save_path) 
    and os.path.exists(y_test_save_path)):
    
    print('-------------Load Datesets-------------------------')
    x_train_save = np.load(x_train_save_path)
    y_train_save = np.load(y_train_save_path)
    x_test_save = np.load(x_test_save_path)
    y_test_save = np.load(y_test_save_path)
    x_train= np.reshape(x_train_save, (len(x_train_save), 28, 28))
    x_test = np.reshape(x_test_save, (len(x_test_save), 28, 28))
else:
    print('------------Generate Data sets-------------------')
    x_train, y_train = generateds(train_path, train_txt)
    x_test, y_test = generateds(test_path, test_txt) 
    
    print('-------------Save Datasets-----------------')
    x_train_save = np.reshape(x_train, (len(x_train), -1)) 
    x_test_save = np.reshape(x_test, (len(x_test), -1))
    np.save(x_train_save_path, x_train_save)
    np.save(y_train_save_path, y_train)
    np.save(x_test_save_path, x_test_save)
    np.save(y_test_save_path, y_test)
    
    
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1)
model.summary()

------------Generate Data sets-------------------
loading :28755_0.jpg 0

loading :13360_5.jpg 5

loading :57662_5.jpg 5

loading :21455_5.jpg 5

loading :59351_5.jpg 5

loading :39461_3.jpg 3

loading :22720_5.jpg 5

loading :52282_6.jpg 6

loading :10872_3.jpg 3

loading :17077_3.jpg 3

loading :23650_8.jpg 8

loading :56239_0.jpg 0

loading :26079_8.jpg 8

loading :52645_6.jpg 6

loading :12727_2.jpg 2

loading :41052_0.jpg 0

loading :9148_4.jpg 4

loading :53908_1.jpg 1

loading :37061_1.jpg 1

loading :1645_0.jpg 0

loading :28683_4.jpg 4

loading :53318_6.jpg 6

loading :29761_4.jpg 4

loading :12201_6.jpg 6

loading :17042_1.jpg 1

loading :39117_4.jpg 4

loading :16573_5.jpg 5

loading :7726_4.jpg 4

loading :49029_2.jpg 2

loading :14651_1.jpg 1

loading :29343_5.jpg 5

loading :27726_3.jpg 3

loading :24053_9.jpg 9

loading :58584_0.jpg 0

loading :18713_2.jpg 2

loading :59958_1.jpg 1

loading :47646_5.jpg 5

loading :18536_6.jpg 6

loading :7352_7.jpg 7

loading :20878_5.j

KeyboardInterrupt: 

### 2、数据增强，增大数据量

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageGenerator


mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(x_train.shape[0], 28, 28 1) # 给数据增加一个维度，使数据和网络结构匹配
x_test= x_test.reshape(x_test.shape[0], 28, 28, 1)


model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test), validation_freq=1)
model.summary()

#### 2.1、数据增强（增大数据量）

#### 2.2、数据增强可视化 (代码 show_augmented _images.py)

### 3、断点续训，存取模型

#### 3.1、读取模型

#### 3.2、保存模型

### 4、参数提取，写至文本

#### 4.1、提取可训练参数

#### 4.2、设置 print 输出格式

### 5、acc/loss 可视化，查看效果

#### 5.1、acc 曲线与 loss 曲线

### 6、应用程序，给图识物

#### 6.1、给图识物

#### 6.2、前向传播执行应用

## 三、补充

### 1、数据集 Fashion_mnist